In [144]:
def getDigits(num):
    digits = []
    while num > 0:
        d = num%10
        num = num//10
        digits.append(d)
    return digits

def repackInstr(op):
    dig = getDigits(op)
    instr = [0,0,0,0]
    n = 0
    for i in dig:
        if (n==0): instr[0] = i
        elif (n==1): instr[0] += 10*i
        else: instr[n-1] = i
        n += 1
    return instr

In [200]:
class amplifier():
    """Amplifier"""

    def __init__(self, digits, phase, name):
        self.name = name
        self.phase = phase
        self.ip = 0
        self.digits = list(digits)
        self.output = -1
        self.iinp = 0
        self.isRunning = True
        self.isHalted = False
        
    def getOutput(self):
        return self.output
    
    def runProgram(self,theInput):
        if self.isHalted:
            self.isRunning = False
            return
        self.isRunning = True
        inputs = [self.phase,theInput]
        outputDig = 0
        #if (self.ip and self.isRunning):
        #    print("Resuming Amp",self.name,"operation at instruction",self.ip,"with input",theInput)
        while ( self.ip < len(self.digits) and self.isRunning ):
            op_ = self.digits[self.ip]
            if op_ == 99: 
                #print("Amp",self.name,"halted - Final output =",self.output)
                self.isRunning = False
                self.isHalted = True
                self.isRunning
                break
            p = repackInstr(op_)
            op = p[0]
            if op == 3:
                inputDig = inputs[self.iinp]
                self.iinp=1
                a = self.digits[self.ip+1]
                if p[1]:
                    self.digits[ self.digits[a] ] = inputDig
                else:
                    self.digits[ a ] = inputDig
                self.ip += 2
            elif op==4:
                a = self.digits[self.ip+1]
                if p[1]:
                    outputDig = a
                else:
                    outputDig = self.digits[a]
                self.ip += 2
                self.output = outputDig 
                self.isRunning = False
                #print("Amp",self.name,"is waiting for new input at instruction",self.ip,"- Current output =",self.output)
            elif op==5:
                a, b = self.digits[self.ip+1:self.ip+3]
                if p[1]:
                    a_ = a
                else:
                    a_ = self.digits[a]
                if p[2]:
                    b_ = b
                else:
                    b_ = self.digits[b]
                if a_:
                    self.ip = b_
                else:
                    self.ip += 3
            elif op==6:
                a, b = self.digits[self.ip+1:self.ip+3]
                if p[1]:
                    a_ = a
                else:
                    a_ = self.digits[a]
                if p[2]:
                    b_ = b
                else:
                    b_ = self.digits[b]
                if not a_:
                    self.ip = b_
                else:
                    self.ip += 3
            elif op==7:
                a, b, c = self.digits[self.ip+1:self.ip+4]
                if p[1]:
                    a_ = a
                else:
                    a_ = self.digits[a]
                if p[2]:
                    b_ = b
                else:
                    b_ = self.digits[b]
                if a_ < b_:
                    self.digits[c] = 1
                else:
                    self.digits[c] = 0
                self.ip += 4
            elif op==8:
                a, b, c = self.digits[self.ip+1:self.ip+4]
                if p[1]:
                    a_ = a
                else:
                    a_ = self.digits[a]
                if p[2]:
                    b_ = b
                else:
                    b_ = self.digits[b]            
                if a_ == b_:
                    self.digits[c] = 1
                else:
                    self.digits[c] = 0
                self.ip += 4
            elif op == 1 or op == 2: 
                a, b, c = self.digits[self.ip+1:self.ip+4]            
                if p[1]:
                    a_ = a 
                else:
                    a_ = self.digits[a]    
                if p[2]:
                    b_ = b
                else:
                    b_ = self.digits[b]       
                if op == 1:
                    self.digits[c] = a_ + b_
                if op == 2:
                    self.digits[c] = a_ * b_
                self.ip += 4
            else:
                print("Illegal operator:",op)
                self.isRunning = False
                return self.isRunning
        return self.isRunning

In [201]:
def amploop(prog,phase,inp):
    
    prog_ = list(prog)
    phase_ = list(phase)

    A = amplifier(prog_,phase_[0],"A")
    B = amplifier(prog_,phase_[1],"B")
    C = amplifier(prog_,phase_[2],"C")
    D = amplifier(prog_,phase_[3],"D")
    E = amplifier(prog_,phase_[4],"E")
    
    iA=inp

    while(True):
        if not A.isHalted: 
            A.runProgram(iA)
        iB = A.getOutput()
        if not B.isHalted: 
            B.runProgram(iB)
        iC = B.getOutput()
        if not C.isHalted: 
            C.runProgram(iC)
        iD = C.getOutput()
        if not C.isHalted: 
            D.runProgram(iD)
        iE = D.getOutput()
        if not E.isHalted: 
            E.runProgram(iE)
        else: 
            output = E.getOutput()
            break
        iA = E.getOutput()
    
    return output

In [202]:
testprog = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
testphase = [9,8,7,6,5]
print( amploop(testprog,testphase,0) )

139629729


In [203]:
testprog = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]
testphase = [9,7,8,5,6]
print( amploop(testprog,testphase,0) )

18216


In [204]:
fullprog = [3,8,1001,8,10,8,105,1,0,0,21,38,55,64,89,114,195,276,357,438,99999,3,9,101,3,9,9,102,3,9,9,1001,9,5,9,4,9,99,3,9,101,2,9,9,1002,9,3,9,101,5,9,9,4,9,99,3,9,101,3,9,9,4,9,99,3,9,1002,9,4,9,101,5,9,9,1002,9,5,9,101,5,9,9,102,3,9,9,4,9,99,3,9,101,3,9,9,1002,9,4,9,101,5,9,9,102,5,9,9,1001,9,5,9,4,9,99,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,99]

In [205]:
testphase = [9,7,8,5,6]
print( amploop(fullprog,testphase,0) )

37930920


In [207]:
from sympy.utilities.iterables import multiset_permutations
import numpy as np

results = []
outputmax = 0

a = np.array([9, 8, 7, 6, 5])

for p in multiset_permutations(a):
    fullprog_  = list(fullprog)
    testphase_ = list(p)
    output = amploop(fullprog_,testphase_,0)
    results.append(output)
    if output > outputmax:
        outputmax = output
        phasemax = testphase_

print("Max thrust signal =",outputmax,"for phase",phasemax)

Max thrust signal = 61019896 for phase [7, 8, 9, 5, 6]
